# Problem
You have a function f(a,b,c,d) and want to optimize for a subset of these four variables, say c and d.

A suitable solution to this is to define a function f(c,d), in which a and b are not input arguments but instead are predetermined.

## Enter the decorator

A decorator is a wrapper function that is passed an input function and returns a output function. Look here for more information: http://simeonfranklin.com/blog/2012/jul/1/python-decorators-in-12-steps/. Basically it wraps around the function that you pass to it.

If you want the decorator itself to accept arguments, you add another function layer, three in total.

### The decorator function below accepts:
1. an input function
2. a dictionary with static variables and their values
The decorator returns the same function, but with the static variables removed from the input arguments.

So if we pass the input function f(a,b,c,d) with static variables a=1 and b=2, the decorator will return f(c,d).

In [245]:
# decorator with arguments
def decorator(static_variables):
    def real_decorator(function):
        def wrapper(**dynamic_variables):
            # combine static and dynamic variables to a single dict, to be passed as keyword arguments to function.
            all_variables = static_variables.copy()
            all_variables.update(dynamic_variables)
            return function(**all_variables)
        return wrapper
    return real_decorator

Let's test it out:

In [246]:
def objective_function_a(a, b, c, d):
    return a + b + c + d

static_variables = {'a': 1, 'b': 2}

func_passed_to_optimizer = decorator(static_variables)(objective_function_a)


print func_passed_to_optimizer(**dynamic_variables)
print func_passed_to_optimizer(c=10, d=100)

3
113


## @ syntax
Decorators can also attached to a function in the following way:

In [247]:
static_variables = {'a': 1, 'b': 2}

@decorator(static_variables)
def objective_function_b(a, b, c, d):
    return a + b + c + d

print objective_function_b(c=1, d=100)

104


## non-keyword arguments

The function that is returned by the decorator above needs its arguments in keyword format, i.e. func(c=1, d=100). The following decorator takes non-keyword arguments, so you can call it like this: func(1, 100).

In [248]:
import inspect

# decorator with arguments
def decorator(static_variables):
    def real_decorator(function):
        print "decorator received function '{}' ".format(function.__name__)
        print "with local variables: {}".format(function.__code__.co_varnames)
        print "and arguments: {}".format(inspect.getargspec(function).args)
        function_argument_variable_names = inspect.getargspec(function).args
        wrapper_dynamic_var_names = [name for name in function_argument_variable_names if name not in static_variables.keys()]
        print "Static variables are: {}".format(static_variables.items())
        print "New input arguments for '{}': {}".format(function.__name__, wrapper_dynamic_var_names)
        def wrapper(*wrapper_dynamic_var_names):
            # combine static and dynamic variables to a single dict, to be passed as keyword arguments to function.
            all_variables = static_variables.copy()
            all_variables.update(dynamic_variables)            
            return function(**all_variables)
        wrapper.__name__ = "{}_primed".format(function.__name__)
        return wrapper
    return real_decorator

Testing time:

In [254]:
def objective_function(a, b, c, d):
    g=1
    return a + b + c + d

staticvars = {'a':1, 'b':2}

func = decorator(staticvars)(objective_function)

print "function name: {}".format(func.__name__)

print func(2,2)

decorator received function 'objective_function' 
with local variables: ('a', 'b', 'c', 'd', 'g')
and arguments: ['a', 'b', 'c', 'd']
Static variables are: [('a', 1), ('b', 2)]
New input arguments for 'objective_function': ['c', 'd']
function name: objective_function_primed
3


Now with the more common @ syntax:

In [256]:
staticvars = {'a':1, 'b':2}

@decorator(staticvars)
def objective_function(a, b, c, d):
    return (a + b + c) ** d

print objective_function(2, 2)

decorator received function 'objective_function' 
with local variables: ('a', 'b', 'c', 'd')
and arguments: ['a', 'b', 'c', 'd']
Static variables are: [('a', 1), ('b', 2)]
New input arguments for 'objective_function': ['c', 'd']
1


Let us try to minimize f(a,b,c,d) using d as a free variable:

In [257]:
import numpy as np
from scipy.optimize import minimize

def objective_function(a, b, c, d):
    return (a + b + c) ** d

staticvars = {'a':1, 'b':2, 'c':10}
objective_function_var_d = decorator(staticvars)(objective_function)

print objective_function_var_d.__code__.co_varnames

d0 = np.array([1.3])
res = minimize(objective_function_var_d, d0, method='nelder-mead',
                options={'xtol': 1e-8, 'disp': True})

decorator received function 'objective_function' 
with local variables: ('a', 'b', 'c', 'd')
and arguments: ['a', 'b', 'c', 'd']
Static variables are: [('a', 1), ('c', 10), ('b', 2)]
New input arguments for 'objective_function': ['d']
('wrapper_dynamic_var_names', 'all_variables')
Optimization terminated successfully.
         Current function value: 1.000000
         Iterations: 24
         Function evaluations: 71
